# Autorepairer Documentation

## Description

All of the code that i wrote for the automatic repairer is in the folder 'autorepair'.
The tests are implemented in the test folder and the main.py was used to call the tests.

The debug_and_repair() method is defined in ./autorepair/debugger.py.
The only change i did, compared to the simple_debug_and_repair() from the project description, was the Mutator class.

The new mutator class (AllMightySuperMutator in ./autorepair/mutators/allmighty_super_mutator.py) subclasses the ConditionMutator and if a node should be swapped it randomly picks and executes one of several custom swap methods (including the one from the original ConditionMutator).
In each mutation i apply only one swap method, which again only apply one change at a time. This makes things a lot easier and more predictable.
Also i rewrote the Visitor (AllMightyVisitor) that collects conditions and definitions in expectation, that i will need to collect more than that, but that was not the case. So this is actually the same as the visitor of the ConditionMutator.

In [ ]:
from random import Random
import random
import ast
from ast import NodeVisitor
import astor
import copy
from copy import deepcopy
from debuggingbook.Repairer import StatementMutator, ConditionMutator

rand = Random()
rand.seed(42)

In [ ]:
def get_all_mutation_sources(trees, tp=None):
    """Return all conditions from the AST (or AST list) `trees`.
    If `tp` is given, return only elements of that type."""
    if not isinstance(trees, list):
        assert isinstance(trees, ast.AST)
        trees = [trees]

    visitor = AllMightyVisitor()
    for tree in trees:
        visitor.visit(tree)
    conditions = visitor.conditions
    definitions = visitor.definitions
    if tp is not None:
        conditions = [c for c in conditions if isinstance(c, tp)]
        definitions = [c for c in definitions if isinstance(c, tp)]

    return conditions, definitions


class AllMightyVisitor(NodeVisitor):
    def __init__(self):
        self.conditions = []
        self.conditions_seen = set()
        self.definitions = []
        self.definitions_seen = set()
        super().__init__()

    def add_to(self,target_str, node, attr=None):
        """ add the attribute of a node to the target list of seen attributes """
        target = getattr(self, target_str)
        target_seen = getattr(self, target_str + "_seen")

        if attr:
            elems = getattr(node, attr, [])
        else:
            elems = node
        if not isinstance(elems, list):
            elems = [elems]
        for elem in elems:
            elem_str = astor.to_source(elem)
            if elem_str not in target_seen:
                target.append(elem)
                target_seen.add(elem_str)

    def visit_FunctionDef(self, node):
        self.add_to('definitions', node)
        return super().generic_visit(node)

    def visit_AsyncFunctionDef(self, node):
        self.visit_FunctionDef(node)

    def visit_ClassDef(self, node):
        self.visit_FunctionDef(node)

    def visit_BoolOp(self, node):
        self.add_to('conditions', node, 'values')
        return super().generic_visit(node)

    def visit_Name(self, node):
        self.add_to('definitions', node)

    def visit_UnaryOp(self, node):
        if isinstance(node.op, ast.Not):
            self.add_to('conditions', node, 'operand')
        return super().generic_visit(node)

    def generic_visit(self, node):
        if hasattr(node, 'test'):
            self.add_to('conditions', node, 'test')
        return super().generic_visit(node)


In [ ]:
class AllMightySuperMutator(ConditionMutator):
    """Mutate all kinds of stuff in an AST"""

    def __init__(self, *args, **kwargs):
        """Constructor. Arguments are as with `StatementMutator` constructor."""
        super().__init__(*args, **kwargs)
        self.conditions, self.definitions = get_all_mutation_sources(self.source)

    def choose_condition(self):
        """Return a random condition from source."""
        return copy.deepcopy(random.choice(self.conditions))

    def choose_name(self):
        """ Return a random name from source. """
        return copy.deepcopy(random.choice(self.names))

    def choose_bool_op(self):
        return random.choice(['set', 'not', 'and', 'or'])


## Swap methods


In [ ]:
    def swap(self, node):
        # i dont know why, but i need another random instance or things stop
        # working
        swap_func_name = "swap_" + rand.choice(['condition',
            'one_off_error','two_occurrences', 'operator',
            'definition_and_occurrences'])
        swap_func = getattr(self, swap_func_name)
        return swap_func(node)
        #  return self.swap_condition(node)


The main swap method picks one of the following defined swap_ methods at random and executes it:

### swap_condition

In [ ]:
    def swap_condition(self, node):
        return ConditionMutator.swap(self, node)

Calls the original ConditionMutator.swap() method.

### swap_one_off_error

First of all, in order to find local definitions and occurrences of variables i implemented a node visitor, that collects all ast.Name instances classified by their type(load or store). We need this for a few of our swap methods, that substitude or swap variable names.

In [ ]:
class NameVisitor(NodeVisitor):

    def __init__(self):
        self.load = []
        self.store = []

    def visit_Name(self, node):
        if isinstance(node.ctx,ast.Load)==True:
            self.load.append(node)
        if isinstance(node.ctx,ast.Store)==True:
            self.store.append(node)

        return super().generic_visit(node)
    
    
    def swap_definition_and_occurrences(self, node):
        node = deepcopy(node)
        visitor = NameVisitor()
        visitor.visit(node)
        store = visitor.store
        load = visitor.load
        if not store or (len(load)<1):
            return self.swap(node)
        # get first definition in node
        store_target = store[0] # take the first, not random
        # filter out all occurrences of same name as store_target
        load_targets = [name for name in load if name.id == store_target.id]
        #  print(names)
        new_id = "fixed_by_autorepair_var"
        store_target.id = new_id
        for load_target in load_targets:
            load_target.id = new_id
        return node

Then, in order to make it simpler to actually swap the complete name instances without knowing what the overlying context was, i wrote some helper functions. 

mark_parents() recursively goes over the ast and marks at each node its parent. 

get_child_attr_name() returns the attribute name of a root node for a parent node. So if i have a name instance, that i know is child of some root node, but i dont know in what context that name appeared, this function finds that out. 

setattr() overwrites some child of a given root node with another object.

This whole proces comes in handy, when i want to swap a child name instance of a root node with something else without having a handle to the parent of the name instance itself.

In [ ]:
class AllMightySuperMutator(ConditionMutator):

    def _mark_parents(self, root):
        for node in ast.walk(root):
            for child in ast.iter_child_nodes(node):
                child.parent = node
                #  if  not type(child) in (int, float, bool, str):
                self._mark_parents(child)
        return node

    def _get_child_attr_name(self, parent, child):
        for _child in dir(parent):
            if getattr(parent,_child) is child:
                return _child

    def _setattr(self, node, target, expr):
        self._mark_parents(node)
        attr = self._get_child_attr_name(target.parent, target)
        #  print(attr)
        if(attr):
            setattr(target.parent, attr, expr)

Now, what swap_one_off_error does is randomly picking one occurence of a variable x and increment or decrement it(Again randomly picked).
So we generate the ast of an expression from "n+1" or "n-1" and change the name to the randomly picked variable.
Then we substitute the name instance with this generated expression (the body of it, since its surrounded by an ast.Module instance and we only want the BoolOp value).

In [ ]:
    def swap_one_off_error(self, node):
        node = deepcopy(node)
        # get all names, with ctx Load
        visitor = NameVisitor()
        visitor.visit(node)
        names = visitor.load
        if not names:
            return self.swap(node)
        # choose a random name from definitions to increment or decrement
        target = rand.choice(names)
        target_id = target.id
        expr = rand.choice(["n+1","n-1"])
        expr = ast.parse(expr).body[0].value
        expr.left.id = target_id
        # now exchange n with the chosen name and replace name with that
        self._setattr(node, target, expr)
        return node


This solves one off errors, for example in loop iterators or return statements etc.

### swap_definition_and_occurrences
 This swap method takes the first definition of some local variable and swaps the id of it and all its occurrences by something new, in this case "fixed_by_autorepair_var".

In [ ]:
    def swap_definition_and_occurrences(self, node):
        node = deepcopy(node)
        visitor = NameVisitor()
        visitor.visit(node)
        store = visitor.store
        load = visitor.load
        if not store or (len(load)<1):
            return self.swap(node)
        # get first definition in node
        store_target = store[0] # take the first, not random
        # filter out all occurrences of same name as store_target
        load_targets = [name for name in load if name.id == store_target.id]
        #  print(names)
        new_id = "fixed_by_autorepair_var"
        store_target.id = new_id
        for load_target in load_targets:
            load_target.id = new_id
        return node


As in test factorial2(), this takes care of variables that get overwritten by local variables with the same name by mistake.

### swap_two_occurrences

This swap method simply picks two names at random and swaps their id.
It is important that we actually pick two different names, or the outcome will be the same as before, which slows down the repairer.

In [ ]:
    def swap_two_occurrences(self, node):
        #  if isinstance(node, ast.Return):
            #  return self.swap(node)
        node = deepcopy(node)
        #  # get all names, with ctx Load
        visitor = NameVisitor()
        visitor.visit(node)
        names = visitor.load
        #  #  print(names)
        #  #  print("COLLECTED ALL NAMES")
        if not names or len(names) < 2:
            return self.swap(node)
        #  print("before:")
        #  print( astor.to_source(node))
        #  print(f"names:{[name.id for name in names]}")
        target1 = rand.choice(names)
        target2 = rand.choice(names)
        # as long as drawing the same name redraw
        while (target1.id == target2.id):
            target2 = rand.choice(names)
        # swap the two names in the node
        h = deepcopy(target1.id)
        target1.id = target2.id
        target2.id = h
        #  print("after:")
        #  print( astor.to_source(node))
        return node


### swap_operator

This swap method randomly picks one operator (like +, -, \*) and substitutes it with another operator.
For this there is a simple OperatorVisitor, that returns all instances of operators from binary operations and augmented assignments in a root node.

In [ ]:
class OperatorVisitor(NodeVisitor):

    def __init__(self):
        self.operators = []

    def visit_BinOp(self, node):
        self.operators.append(node)
        return super().generic_visit(node)

    def visit_AugAssign(self, node):
        return self.visit_BinOp(node)


In [ ]:
    def swap_operator(self, node):
        node = deepcopy(node)
        visitor = OperatorVisitor()
        visitor.visit(node)
        operators = visitor.operators
        if not operators:
            return self.swap(node)
        # choose a random operator to swap
        target = rand.choice(operators)
        operators = [ast.Add ,ast.Mult]
        target.op = rand.choice(operators)()
        return node
